<a href="https://colab.research.google.com/github/Lisker2/CS309_Project/blob/main/Tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/nadavbra/shared_utils.git
!git clone https://github.com/nadavbra/protein_bert
!git clone https://github.com/khanhlee/bert-enhancer
!git clone https://github.com/Lisker2/CS309_Project 

In [ ]:
# first install python 3.6
!sudo apt-get update -y
!sudo apt-get install python3.6
# change alternatives
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 1
# select python version
!sudo update-alternatives --config python3
# check python version
!python --version
# install pip for new python 
!sudo apt-get install python3.6-distutils
!wget https://bootstrap.pypa.io/get-pip.py
!python get-pip.py
# upgrade pip
!sudo apt install python3-pip
!python -m pip install --upgrade pip
!python3.6 -m pip install tensorflow==2.4.0
!python3.6 -m pip install protein-bert
!python3.6 -m pip install matplotlib
!python3.6 -m pip  install scikit-learn
!python3.6 -m pip  install numpy
!python3.6 -m pip  install pandas
!python3.6 -m pip  install scipy
!python3.6 -m pip  install ipykernel

In [28]:
!python3.6 /content/CS309_Project/code_py/tune.py

2023-05-05 00:38:48.907897: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
sequence_length 202
[2023_05_05-00:38:52] Training set: Filtered out 0 of 2768 (0.0%) records of lengths exceeding 200.
[2023_05_05-00:38:52] Validation set: Filtered out 0 of 300 (0.0%) records of lengths exceeding 200.
[2023_05_05-00:38:52] Training with frozen pretrained layers...
2023-05-05 00:38:52.299541: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-05-05 00:38:52.299976: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-05-05 00:38:52.300046: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-05 00:38:52.3000